In [ ]:
import os
import sys
import random
import pathlib

# processing
import geojson
import utm
import shapefile
import shapely
import shapely.geometry

# interactive/visualization
import folium
from tqdm.notebook import tqdm
from IPython.display import display

In [ ]:
def utm2lonlat(c):
    # the shapefile coordinate system is UTM zone 47N (northern hemisphere)
    lat, lon = utm.to_latlon(*c, 47, northern=True, strict=False)
    return (lon, lat)

shapefile: [shp-f60](http://www.forest.go.th/land/%e0%b8%a3%e0%b8%b2%e0%b8%a2%e0%b8%87%e0%b8%b2%e0%b8%99%e0%b9%82%e0%b8%84%e0%b8%a3%e0%b8%87%e0%b8%81%e0%b8%b2%e0%b8%a3%e0%b8%88%e0%b8%b1%e0%b8%94%e0%b8%97%e0%b8%b3%e0%b8%82%e0%b9%89%e0%b8%ad%e0%b8%a1-5/)

contains the _entire_ country's forest regions

In [ ]:
# read the shapefile
# stole from https://gist.github.com/frankrowe/6071443

# set path here
SHAPEFILE_PATH='/mnt/ntfs/shp_f60/f60.shp'
shpfile = str(pathlib.Path(SHAPEFILE_PATH).resolve())

reader = shapefile.Reader(shpfile)
fields = reader.fields[1:]
field_names = [field[0] for field in fields]

def process_sr(sr):
    atr = dict(zip(field_names, sr.record))
    geom = sr.shape.__geo_interface__
    pointfix = geojson.utils.map_tuples(utm2lonlat, geom)
    return dict(type="Feature", geometry=pointfix, properties=atr)

process_sr(reader.shapeRecord(29))

In [ ]:
buffer = []
for srindex in tqdm(range(10000)):
    sr = reader.shapeRecord(srindex)
    buffer.append(process_sr(sr))

In [ ]:
# skip
#with open('f60.geojson', 'r') as f:
#    f60 = geojson.load(f)

In [ ]:
# also skip
#f60_buffer = geojson.FeatureCollection(buffer)
#with open('f60.geojson', 'w') as f:
#    geojson.dump(f60_buffer, f)

In [ ]:
def stylefunc(f):
    lo = 50
    hi = 255
    r = random.randint(lo, hi)
    g = random.randint(lo, hi)
    b = random.randint(lo, hi)
    fill_color = "#{:02x}{:02x}{:02x}".format(r,g,b)
    return {
        "color": fill_color,
        "fillColor": fill_color
    }

stylefunc(None)

# _ป่าควนเคร็ง~~อยู่ไหนวะสัส~~ GET !!!_

_ไม่ใช่ก็ใกล้เคียงละวะ_

-----

bounding box หยาบๆ

    (1)---(2)
     |     |
     |     |
    (4)---(3)

    latlon
    1) 8.132766,99.977713
    2) 8.129368,100.220099
    3) 7.713491,100.225925
    4) 7.715785,99.974589

In [ ]:
# (lon, lat) format!
kuankreng = shapely.geometry.Polygon([
    (99.977710,8.132766), (100.220099,8.129368),
    (100.225925,7.713491), (99.974589,7.715785)])

roi = []
for forest in buffer:
    fshape = shapely.geometry.shape(forest['geometry'])
    if kuankreng.contains(fshape):
        roi.append(forest)

#roi_geojson = geojson.MultiPolygon(shp['geometry']['coordinates'] for shp in roi)
roi_geojson = geojson.FeatureCollection(roi)
with open('kuankreng.geojson', 'w') as gjfile:
    geojson.dump(roi_geojson, gjfile)

In [ ]:
lmap = folium.Map(location=[7.89, 100.10], zoom_start=6, tiles="Stamen Terrain")

b_drawn = folium.FeatureGroup(name="geojson")

for b in tqdm(roi):
    forest_shp = folium.GeoJson(b, 
                   tooltip=b['properties']['OBJECTID'],
                   style_function=stylefunc
                  )
    #id_popup = folium.Popup(str(b['properties']['OBJECTID']), sticky=True)
    #id_popup.add_to(forest_shp)
    forest_shp.add_to(b_drawn)

b_drawn.add_to(lmap)

# DO NOT ATTEMPT TO DRAW IT RAW, YOU HAVE BEEN WARNED.
#folium.GeoJson(f60, name='geojson_full', style_function=stylefunc).add_to(lmap)

folium.GeoJson(roi_geojson, name="kuankreng").add_to(lmap)

kuankreng_geo = shapely.geometry.mapping(kuankreng)
folium.GeoJson(kuankreng_geo, name="est. bbox").add_to(lmap)

folium.TileLayer(tiles="cartodb positron").add_to(lmap)
folium.LayerControl().add_to(lmap)

ipdisp(lmap)

In [ ]:
with open(str(pathlib.Path('./kuankreng.geojson')), 'r') as gj:
    kuankreng = geojson.load(gj)

shp_features = kuankreng['features']
shp_shapely = [ shapely.geometry.shape(f['geometry']).buffer(0) for f in shp_features ]

shp_all = shapely.geometry.MultiPolygon(shp_shapely)
display(shp_all)
